 <font color="#339966" face="Segoe UI Semibold" size="5">Начало:</font>

In [1]:
# Необходимо установить следующие пакеты

#!pip install distance
#!pip install progressbar2

In [2]:
#!/usr/bin/env python3
# @hidden_cell
#coding: utf-8
import datetime, time
import math as m
from IPython.display import Javascript
from IPython.display import display
from IPython.display import HTML
import sys, os
import warnings
import numpy as np
import pandas as pd
import progressbar #необходимо установить
from math import radians, cos, sin, asin, sqrt
from datetime import timedelta
import requests, json
import winsound
from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import DBSCAN
import codecs, difflib, distance #необходимо установить distance

In [3]:
def Way(lat_Start,lon_Start,lat_Finish, lon_Finish,ID,Step,TIME):
    """
    Процедура для эмуляции html-страницы
    """
    
    js2 = """
    var start_point = ["""+lat_Start+""", """+lon_Start+"""];
    var end_point = ["""+lat_Finish+""", """+lon_Finish+"""];
    ymaps.ready(function () {
    var theEl=document.getElementById("output");    
    ymaps.route([start_point, end_point], {
            mapStateAutoApply: false,
            avoidTrafficJams: false,
            multiRoute: false,
            routingMode: "auto",
            viaIndexes: []
        }).then(function (route) {
            var points = route.getWayPoints();  
            theEl.innerHTML = theEl.innerHTML+'"""+str(ID)+"""'+'|'+start_point+'; '+end_point+'|Расстояние --'+route.getHumanLength() + '|'+route.getLength()+'|'+ route.getJamsTime() + '|'+ route.getTime()+ '|'+"""+str(Step)+"""+'|'+"""+str(TIME)+"""+'\\r\\n';
        }, function (error) {
        theEl.innerHTML = theEl.innerHTML+'"""+str(ID)+"""'+'||||||' +"""+str(Step)+"""+'|'+"""+str(TIME)+"""+'\\r\\n';
    });
    });
    """
    jj=Javascript(js2)
    display(jj)
    
def haversine(lonlat1, lonlat2):
    """
    Вычисляет прямое расстояние между двумя точками на карте на основании координат
    """
    lat1, lon1 = lonlat1
    lat2, lon2 = lonlat2
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Средний радиус Земли в километрах
    return c * r

In [4]:
YandexHTTPGeocoderKey = '23aa748e-b261-49a3-8e25-a353715fdb8c' # API клюс Яндекса категории "JavaScript API и HTTP Геокодер"
YandexHTTPGeocoderKey2 ='ad87feb5-1f0e-4a8c-8c4e-3f8718d0a22e' # API клюс Яндекса категории "JavaScript API и HTTP Геокодер" Ксюша

YandexHTTPGeocoderKey4 = 'cf1818ff-f029-408e-8a91-e4e223ad6693' # API клюс Яндекса категории "JavaScript API и HTTP Геокодер" Мама, na.tolyupa

s="""
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">
<HEAD>
    <TITLE>Заголовок</TITLE>
    <META HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=windows-1251">
</head>

<script src="https://api-maps.yandex.ru/2.1/?apikey="""+str(YandexHTTPGeocoderKey4)+"""&lang=ru_RU"></script>
<script type="text/javascript" src="jquery-1.3.1.min.js" ></script>
<script type="text/javascript">  
    var myOutput=document.getElementById("downloadfile")

</script>

<body>
    <div hidden id="output"></div>
    <div id="downloadfile"></div>
</body>
"""

 <font color="#339966" face="Segoe UI Semibold" size="5">Входные данные:</font>

In [5]:
СollateralFile = "A_part1.txt" # файл, где хранятся данные об Объекте залога
SberbankAdress = "SBPointMSK.txt" # файл, где хранятся данные о рабочих адресах сотрудников ПМЗ
TimeSleep = 0.05 # Пауза при обрадотка данных, в секундах <font color="#339966" face="Segoe UI" size="4">Обработка данных:</font>

In [6]:
frame = pd.read_csv(СollateralFile, header=None, sep='|',encoding='utf-8')
sbp = pd.read_csv(SberbankAdress, header=None, sep='|',encoding='cp1251')
frame['index']=frame.index; sbp['index']=sbp.index;
frame['stateSB']=frame['latSB']=frame['lngSB']=frame['latZ']=frame['lngZ']=0
frame['zalogstate']=frame['zalogaddress']=frame['km']=frame['time_jams']=frame['day']=frame['day_num']=frame['day_km']=frame['time_arrival']=frame['accuracy']=frame['quality']=frame['ok']=np.nan
sbp['lat']=sbp['lng']=sbp['rez']=np.nan

frame.rename(columns={0: 'sb', 1: 'zalog',2:'time',3:'num'}, inplace=True)
sbp.rename(columns={0: 'sb'}, inplace=True)

 <font color="#339966" face="Segoe UI Semibold" size="5">Обработка данных:</font>

In [7]:
now = datetime.datetime.now()
print ('Текущее время: ',now.strftime("%Y-%m-%d %H:%M"))

start = datetime.datetime.now()
bar = progressbar.ProgressBar(max_value=frame['sb'].count())

#num_start = 3059
#num_finish = 3663
#i=num_start

i=0 #комментировать если обсчет не всего массива
for row,row1 in zip(frame['zalog'],frame['sb']): #комментировать если обсчет не всего массива
    time.sleep(TimeSleep)
    try:
        g = requests.get('https://geocode-maps.yandex.ru/1.x/?apikey='+YandexHTTPGeocoderKey+'&geocode='+row+'&format=json&results=1')
    except:
        i+=1
        time.sleep(TimeSleep)
        continue
    time.sleep(TimeSleep)
        
    if int(g.json()['response']['GeoObjectCollection']['metaDataProperty']['GeocoderResponseMetaData']['found'])>0:
        for result in range(int(g.json()['response']['GeoObjectCollection']['metaDataProperty']['GeocoderResponseMetaData']['results'])):
            frame.loc[i,'latZ'] = g.json()['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos'].split(" ")[1]
            frame.loc[i,'lngZ'] = g.json()['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos'].split(" ")[0]
            frame.loc[i,'zalogaddress'] = g.json()['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['metaDataProperty']['GeocoderMetaData']['text']
            frame.loc[i,'accuracy'] = g.json()['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['metaDataProperty']['GeocoderMetaData']['precision']
            frame.loc[i,'quality'] = g.json()['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['metaDataProperty']['GeocoderMetaData']['kind']
            try:
                frame.loc[i,'zalogstate'] = g.json()['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['metaDataProperty']['GeocoderMetaData']['Address']['Components'][2]['name']
            except:
                frame.loc[i,'zalogstate'] = '-'
            frame.loc[i,'ok'] = g.json()['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos']   
    i+=1
    bar.update(i)
bar.finish()        
finish = datetime.datetime.now()

print ('Время выполнения: ', (finish-start))#.strftime("%d-%m-%Y %H:%M")

Текущее время:  2022-11-13 11:38


100% (59 of 59) |########################| Elapsed Time: 0:00:19 Time:  0:00:19


Время выполнения:  0:00:19.318123


 <font color="#339966" face="Segoe UI Semibold" size="5">Обработка Адресов Сбербанка:</font>

In [8]:
print ('Текущее время: ', datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))
start = datetime.datetime.now()
bar = progressbar.ProgressBar(max_value=sbp['sb'].count())
i=0
for row in sbp['sb']:
    try:
        g = requests.get('https://geocode-maps.yandex.ru/1.x/?apikey='+YandexHTTPGeocoderKey2+'&geocode='+row+'&format=json&results=1')
    except:
        i+=1
        time.sleep(TimeSleep)
        continue 
        
    if int(g.json()['response']['GeoObjectCollection']['metaDataProperty']['GeocoderResponseMetaData']['found'])>0:
        sbp.loc[i,'lat']=g.json()['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos'].split(" ")[1]
        sbp.loc[i,'lng'] = g.json()['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos'].split(" ")[0]
        sbp.loc[i,'rez'] = g.json()['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['metaDataProperty']['GeocoderMetaData']['precision']
    bar.update(i)
    i+=1
bar.finish() 

finish = datetime.datetime.now()
print ('Время выполнения: ', (finish-start))

  0% (0 of 39) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

Текущее время:  2022-11-13 11:39


100% (39 of 39) |########################| Elapsed Time: 0:00:07 Time:  0:00:07


Время выполнения:  0:00:07.692972


In [9]:
print ('Текущее время: ', datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))
start = datetime.datetime.now()
near = pd.DataFrame([])
near['index']=near['latZ']=near['lngZ']=0;
near['localaddress1']=near['km_s1']=near['lat_s1']=near['lng_s1']=np.nan
near['localaddress2']=near['km_s2']=near['lat_s2']=near['lng_s2']=np.nan
near['localaddress3']=near['km_s3']=near['lat_s3']=near['lng_s3']=np.nan
near['localaddress4']=near['km_s4']=near['lat_s4']=near['lng_s4']=np.nan
near['localaddress5']=near['km_s5']=near['lat_s5']=near['lng_s5']=np.nan
nn=0
for i in range(frame.shape[0]):
    KM=[]
    for j in range(sbp.shape[0]):
        nn+=1
        lat1=float(frame.loc[i,'latZ']); lon1=float(frame.loc[i,'lngZ']); lat2=float(sbp.loc[j,'lat']); lon2=float(sbp.loc[j,'lng']);
        lon1, lat1, lon2, lat2 = map(radians, (lon1, lat1, lon2, lat2))
        a = sin((lat2-lat1)/2) ** 2 + cos(lat1) * cos(lat2) * sin((lon2-lon1)/2) ** 2
        km = 2*asin(sqrt(a))*6367.0
        KM.append((round(km,2),sbp.loc[j,'sb'],sbp.loc[j,'lat'],sbp.loc[j,'lng']))
    KM.sort(key=lambda x: x[0])
    near.loc[i,'index']=frame.loc[i,'index']; near.loc[i,'latZ']=frame.loc[i,'latZ']; near.loc[i,'lngZ']=frame.loc[i,'lngZ'];
    near.loc[i,'km_s1']=float(KM[0][0]); near.loc[i,'localaddress1']=KM[0][1]; near.loc[i,'lat_s1']=KM[0][2]; near.loc[i,'lng_s1']=KM[0][3];
    near.loc[i,'km_s2']=float(KM[1][0]); near.loc[i,'localaddress2']=KM[1][1]; near.loc[i,'lat_s2']=KM[1][2]; near.loc[i,'lng_s2']=KM[1][3];
    near.loc[i,'km_s3']=float(KM[2][0]); near.loc[i,'localaddress3']=KM[2][1]; near.loc[i,'lat_s3']=KM[2][2]; near.loc[i,'lng_s3']=KM[2][3];
    near.loc[i,'km_s4']=float(KM[3][0]); near.loc[i,'localaddress4']=KM[3][1]; near.loc[i,'lat_s4']=KM[3][2]; near.loc[i,'lng_s4']=KM[3][3];
    near.loc[i,'km_s5']=float(KM[4][0]); near.loc[i,'localaddress5']=KM[4][1]; near.loc[i,'lat_s5']=KM[4][2]; near.loc[i,'lng_s5']=KM[4][3];
print (nn)
finish = datetime.datetime.now()
print ('Время выполнения: ', (finish-start))#.strftime("%d-%m-%Y %H:%M")

Текущее время:  2022-11-13 11:39
2301
Время выполнения:  0:00:00.131016


In [10]:
h = HTML(s)
f=display(h)

In [11]:
start = datetime.datetime.now()
timesleep=0.14
print ('Подождите, пожалуйста. Анализ маршрутов займет примерно '+str((timesleep*6*frame['num'].count()+timesleep*0.05)/60.0)+' мин...')

print ('Выполняется шаг 1. Начало '+str(datetime.datetime.now())) 
for latSB,lngSB,latZ,lngZ,index in zip(frame['latSB'],frame['lngSB'],frame['latZ'],frame['lngZ'],frame['index']):
    time.sleep(timesleep) #Пауза полсекунды 
    Way(str(latSB),str(lngSB),str(latZ),str(lngZ),str(index),0,str(datetime.datetime.today().strftime("%Y%m%d%H%M")))

print ('Выполняется шаг 2. Начало '+str(datetime.datetime.now())) 
for latSB,lngSB,latZ,lngZ,index in zip(near['lat_s1'],near['lng_s1'],near['latZ'],near['lngZ'],near['index']):
    time.sleep(timesleep) #Пауза полсекунды 
    Way(str(latSB),str(lngSB),str(latZ),str(lngZ),str(index),1,str(datetime.datetime.today().strftime("%Y%m%d%H%M")))

print ('Выполняется шаг 3. Начало '+str(datetime.datetime.now())) 
for latSB,lngSB,latZ,lngZ,index in zip(near['lat_s2'],near['lng_s2'],near['latZ'],near['lngZ'],near['index']):
    time.sleep(timesleep) #Пауза полсекунды 
    Way(str(latSB),str(lngSB),str(latZ),str(lngZ),str(index),2,str(datetime.datetime.today().strftime("%Y%m%d%H%M")))   

print ('Выполняется шаг 4. Начало '+str(datetime.datetime.now()))     
for latSB,lngSB,latZ,lngZ,index in zip(near['lat_s3'],near['lng_s3'],near['latZ'],near['lngZ'],near['index']):
    time.sleep(timesleep) #Пауза полсекунды 
    Way(str(latSB),str(lngSB),str(latZ),str(lngZ),str(index),3,str(datetime.datetime.today().strftime("%Y%m%d%H%M")))

print ('Выполняется шаг 5. Начало '+str(datetime.datetime.now()))     
for latSB,lngSB,latZ,lngZ,index in zip(near['lat_s4'],near['lng_s4'],near['latZ'],near['lngZ'],near['index']):
    time.sleep(timesleep) #Пауза полсекунды 
    Way(str(latSB),str(lngSB),str(latZ),str(lngZ),str(index),4,str(datetime.datetime.today().strftime("%Y%m%d%H%M")))

print ('Выполняется шаг 6. Начало '+str(datetime.datetime.now()))     
for latSB,lngSB,latZ,lngZ,index in zip(near['lat_s5'],near['lng_s5'],near['latZ'],near['lngZ'],near['index']):
    time.sleep(timesleep) #Пауза полсекунды 
    Way(str(latSB),str(lngSB),str(latZ),str(lngZ),str(index),5,str(datetime.datetime.today().strftime("%Y%m%d%H%M")))  

    
finish = datetime.datetime.now()
RandomName=int(np.random.uniform(0,1)*100000000000000)
print ('Спасибо что ждали. Время выполнения: ', (finish-start))
print ('Данные будут сохранены в файл: '+'Route_'+str(RandomName)+'.txt')
winsound.Beep(1000, 940) #Звуковой сигнал при завершении

Подождите, пожалуйста. Анализ маршрутов займет примерно 0.8261166666666667 мин...
Выполняется шаг 1. Начало 2022-11-13 11:39:20.913455


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Спасибо что ждали. Время выполнения:  0:00:52.851869
Данные будут сохранены в файл: Route_95743581040773.txt


 <font color="#8b0000" face="Segoe UI Semibold" size="7">STOP</font>

In [12]:
#Подготовка ссылки в Jupyter для сохранения найденных расстояний в файл
s10 =  """
<script>
function OutText(text) { 
    myOutput.innerHTML='<a href="data:text/plain;charset=utf-8,%EF%BB%BF'+$(output).html()+'" download="Route_"""+str(RandomName)+""".txt">Сохранить в файл <b>Route_"""+str(RandomName)+""".txt</a>'  
}
</script>
            
<input onclick="OutText('вообще хорошо')" type="button" value="Подготовить сохранение: Route_"""+str(RandomName)+""".txt">  

 """
h10 = HTML(s10)
display(h10)

In [13]:
print (RandomName)
frame_html = pd.read_csv(r'Route_'+str(RandomName)+'.txt', header=None, sep='|')
frame_html.rename(columns={0: 'ID', 1: 'latlng',2:'route-km',3:'route-m',4:'time-jam',5:'time-wo-jam',6:'step',7:'_time'}, inplace=True)

95743581040773


In [14]:
#if (frame_html['ID'].nunique()<=near['num'].nunique()):
merged_inner = pd.merge(left=near, right=frame_html[['ID','route-m','time-jam']].loc[frame_html['step']==0], left_on='index', right_on='ID', how='left')
merged_inner = merged_inner.rename(columns={'ID': 'ID_now', 'route-m': 'route-m_now','time-jam':'time-jam_now'})
    
merged_inner1 = pd.merge(left=merged_inner, right=frame_html[['ID','route-m','time-jam']].loc[frame_html['step']==1], left_on='index', right_on='ID', how='left')
merged_inner1 = merged_inner1.rename(columns={'ID': 'ID_1', 'route-m': 'route-m_1','time-jam':'time-jam_1'})
    
merged_inner2 = pd.merge(left=merged_inner1, right=frame_html[['ID','route-m','time-jam']].loc[frame_html['step']==2], left_on='index', right_on='ID', how='left')
merged_inner2 = merged_inner2.rename(columns={'ID': 'ID_2', 'route-m': 'route-m_2','time-jam':'time-jam_2'})
    
merged_inner3 = pd.merge(left=merged_inner2, right=frame_html[['ID','route-m','time-jam']].loc[frame_html['step']==3], left_on='index', right_on='ID', how='left')
merged_inner3 = merged_inner3.rename(columns={'ID': 'ID_3', 'route-m': 'route-m_3','time-jam':'time-jam_3'})
    
merged_inner4 = pd.merge(left=merged_inner3, right=frame_html[['ID','route-m','time-jam']].loc[frame_html['step']==4], left_on='index', right_on='ID', how='left')
merged_inner4 = merged_inner4.rename(columns={'ID': 'ID_4', 'route-m': 'route-m_4','time-jam':'time-jam_4'})
    
merged_inner5 = pd.merge(left=merged_inner4, right=frame_html[['ID','route-m','time-jam']].loc[frame_html['step']==5], left_on='index', right_on='ID', how='left')
merged_inner5 = merged_inner5.rename(columns={'ID': 'ID_5', 'route-m': 'route-m_5','time-jam':'time-jam_5'})
merged_inner5.drop(['ID_1', 'ID_2','ID_3','ID_4','ID_5'], axis='columns', inplace=True)

RESULT=pd.merge(left=frame, right=merged_inner5, left_on='index', right_on='index')
u=RESULT[['route-m_1','route-m_2','route-m_3','route-m_4','route-m_5']].idxmin(skipna=True,axis=1)
for i in range(u.shape[0]):
    if not(pd.isna(u[i])):
        if u[i] == 'route-m_1': col='localaddress1';
        if u[i] == 'route-m_2': col='localaddress2';
        if u[i] == 'route-m_3': col='localaddress3';
        if u[i] == 'route-m_4': col='localaddress4';
        if u[i] == 'route-m_5': col='localaddress5';
        RESULT.loc[i,'min_route']=RESULT.loc[i,u[i]]
        RESULT.loc[i,'min_address']=RESULT.loc[i,col]
print('Ближайшие рабочие адреса сотрудников ПМЗ определены')
print('Обработано '+str(RESULT.shape[0])+ ' Объектов залога (строк в исходном файле)')
#else: print('Есть дубликаты или ошибка в обработке данных')    

Ближайшие рабочие адреса сотрудников ПМЗ определены
Обработано 59 Объектов залога (строк в исходном файле)


<font color="#339966" face="Segoe UI" size="4">Расчет Лотов</font> 

In [15]:
frame = RESULT[['index','num','sb','stateSB','zalog','zalogstate','zalogaddress','accuracy','quality','ok','route-m_now','time-jam_now','min_route','min_address','localaddress1','route-m_1','time-jam_1','localaddress2','route-m_2','time-jam_2','localaddress3','route-m_3','time-jam_3','localaddress4','route-m_4','time-jam_4','localaddress5','route-m_5','time-jam_5']].copy(deep=True)
frame[['lngZ','latZ']]=frame['ok'].str.split(' ',expand=True)
frame[['latZ','lngZ']] = frame[['latZ','lngZ']].fillna(0)

In [16]:
#Расстояние между объектами
eps_local=10

In [17]:
PolygonM = pd.DataFrame([])
PolygonM['ok']=PolygonM['lat']=PolygonM['lng']=0
PolygonM[['ok','lat','lng']]=frame.loc[frame['latZ']!=0][['ok','latZ','lngZ']].reset_index(drop=True)
PolygonM.dropna(inplace=True)
PolygonM.drop_duplicates(inplace=True)
PolygonM.reset_index(drop=True,inplace=True) 
PolygonM['lat'] = PolygonM['lat'].astype(float)
PolygonM['lng'] = PolygonM['lng'].astype(float)
print ('Найдено уникальных Объектов:',PolygonM.shape[0])

Найдено уникальных Объектов: 56


In [18]:
#Строим матрицу с прямыми расстояниями между объектов
distance_matrix = squareform(pdist(PolygonM[['lat','lng']], (lambda u,v: haversine(u,v))))

In [19]:
# Объединяем в кластеры
db = DBSCAN(eps=eps_local, min_samples=2, metric='precomputed')
y_db = db.fit_predict(distance_matrix)
PolygonM['cluster'] = y_db

In [20]:
for i in range (PolygonM.shape[0]):
    filter_count = PolygonM['cluster'] == PolygonM.loc[i,'cluster']
    PolygonM.loc[i,'uniq_in_cluster']=PolygonM['cluster'].loc[filter_count].count()
if 'uniq_in_cluster' in frame.columns: print ('day is True'); frame.drop(['uniq_in_cluster', 'cluster'], axis='columns', inplace=True)
frame=frame.merge(PolygonM[['ok','cluster','uniq_in_cluster']], how='left',left_on='ok', right_on='ok',indicator=False)

<font color="#339966" face="Segoe UI" size="4">Проверка адресов</font> 

In [21]:
frame['difflib'] = frame.apply(lambda row: difflib.SequenceMatcher(None, str(row['zalog']).lower(), str(row['zalogaddress']).lower()).ratio(), axis=1)
frame['sorensen'] = frame.apply(lambda row: 1 - distance.sorensen(str(row['zalog']).lower(), str(row['zalogaddress']).lower()), axis=1)
frame['jaccard'] = frame.apply(lambda row: 1 - distance.jaccard(str(row['zalog']).lower(), str(row['zalogaddress']).lower()), axis=1)

In [22]:
frame[['index','num','zalog','zalogstate','zalogaddress','accuracy','quality','ok','min_route','min_address','cluster','uniq_in_cluster','difflib','sorensen','jaccard']].to_excel('Результат.xlsx')

<font color="#000066" face="Segoe UI" size="6">Черновик:</font>

[Example]: http://example.com/

Пример ошибочно распознанного адреса [ссылка](https://geocode-maps.yandex.ru/1.x/?apikey=23aa748e-b261-49a3-8e25-a353715fdb8c&geocode=молочно+товарная+ферма+с.+Новые+Чебеньки+Зианчуринского+района+Республики+Башкортостан&format=json&results=4).